<h1> <center> Protección de datos sensibles </center> </h1> 

Este notebook es parte de una guía práctica con algunos lineamientos para tener en cuenta al realizar intercambios de datos o publicar datos que contienen información de carácter sensible. Contiene la aplicación de técnicas presentadas conceptualmente con anterioridad, y propone el uso de herramientas ejemplificadas en el lenguaje Python sin intentar ser una solución completamente abarcativa a problemas de anonimización.

In [1]:
### En caso de que no tengan instalados los siguientes paquetes, se deberán correr las siguientes líneas.
# !pip install scrubadub
# !pip install scrubadub_spacy
# !pip install spacy-transformers

### Scrubadub

Esta [librería](https://scrubadub.readthedocs.io/en/stable/) permite quitar información personal de textos. Esto incluye nombres, emails, urls, usuarios de Twitter o Instagram, fechas de nacimiento, entre otros. Es útil para tratar aquellos campos que tengamos donde se tenga un texto con partes a enmascarar.

In [2]:
import pandas as pd

### Importar librerías y paquetes necesarios para usar scrubadub y spacy
import scrubadub, scrubadub_spacy
import spacy_transformers

### Se importa la librería de hashing
import hashlib

In [3]:
### Se inicializa el scrubber con la localización del país para que reconozca características específicas, como números de teléfono.
scrubber = scrubadub.Scrubber()

### Se agrega el detector elegido
scrubber.add_detector(scrubadub_spacy.detectors.SpacyEntityDetector(model = "es_core_news_md"))

In [4]:
### Se limpia el texto elegido
scrubber.clean("Mi nombre es Juan González")

'Mi nombre es {{NAME}}'

In [5]:
### Se agrega el locale para que reconozca números de teléfono
scrubber = scrubadub.Scrubber(locale='es_AR')
detector = scrubadub_spacy.detectors.SpacyEntityDetector(model='es_core_news_md')
scrubber.add_detector(detector)
text = "contactar a  Emanuel Pereira al 011-81367886"
scrubber.clean(text)
text = "contactar a  Emanuel Pereira al 011 8136 7886"
print(scrubber.clean(text))
text = "contactar a  Emanuel Pereira al 011 81367886"
print(scrubber.clean(text))
text = "contactar a  Emanuel Pereira al 011-8136-7886"
print(scrubber.clean(text))
text = "contactar a  Emanuel Pereira al 011-15-8136-7886"
print(scrubber.clean(text))
text = "contactar a  Emanuel Pereira al 0348-4352206"
print(scrubber.clean(text))
text = "contactar a  Emanuel Pereira al 0348-15-4352206"
print(scrubber.clean(text))

contactar a  {{NAME}} al {{PHONE}}
contactar a  {{NAME}} al {{PHONE}}
contactar a  {{NAME}} al {{PHONE}}
contactar a  {{NAME}} al {{PHONE}}
contactar a  {{NAME}} al {{PHONE}}
contactar a  {{NAME}} al {{PHONE}}


In [6]:
### Revisión de cómo funciona cuando ingresamos emails, usuarios de redes sociales y teléfonos

scrubber.clean("Mi nombre es Juan González y mi mail juan@gmail.com, usuario @juan1. Sitio web: www.sitiojuan.com.arg Mi celular es 011 55550022")

'Mi nombre es {{NAME}} y mi mail {{EMAIL}}, usuario {{TWITTER}}. Sitio web: {{URL}} Mi celular es {{PHONE}}'

### Hashing

Hashlib es una [librería](https://docs.python.org/3/library/hashlib.html) de Python que permite encriptar datos con una serie de distintos algoritmos, detallados en el link. Los algoritmos de hashing son funciones matemáticas usadas para transformar datos en valores o códigos hash y que sea imposible identificar el contenido original de ese dato. El hashing es una operación unidireccional en el que los valores ingresados son transformados a un código hash de longitud finita que no puede revertirse.

In [7]:
### Se crea un dataframe de ejemplo con las jurisdicciones
dic = {'Jurisdicción':['Buenos Aires', 'Córdoba', 'San Juan', 'Buenos Aires', 'Buenos Aires', 'Córdoba', 'Entre Ríos']}
df = pd.DataFrame(data = dic)
df

,Jurisdicción
0,Buenos Aires
1,Córdoba
2,San Juan
3,Buenos Aires
4,Buenos Aires
5,Córdoba
6,Entre Ríos


In [8]:
### Como ejemplo se utiliza un hashing con sha256.
df['Jurisdicción'].apply(lambda x: x.encode()).apply(hashlib.sha256).apply(lambda x: x.hexdigest())

0    abae768240354770b732e87de24a76aa3123c0abbe7f4e...
1    263bcb39796e48285ae952fb25ddb5550242fc8d3bafae...
2    0c754c271376cd8bdaf8e93edba2c6e034afa14925d617...
3    abae768240354770b732e87de24a76aa3123c0abbe7f4e...
4    abae768240354770b732e87de24a76aa3123c0abbe7f4e...
5    263bcb39796e48285ae952fb25ddb5550242fc8d3bafae...
6    b50525cee17c74a8416f580e965be92058b7792a3ebf46...
Name: Jurisdicción, dtype: object